In [1]:
import pandas as pd
import numpy as np
import os
import copy
from PopSynthesis.Methods.connect_HH_PP.scripts.sample_hh_main import *
from PopSynthesis.Methods.connect_HH_PP.scripts.process_all_hh_pp import *    

In [2]:
def wrapper_get_pool(have_the_file=False):
    if have_the_file:
        pool = pd.read_csv("./pool_all_samples.csv")
    else:
        df_seed = pd.read_csv(os.path.join(processed_data, "ori_sample_hh.csv"))
        # drop all the ids as they are not needed for in BN learning
        id_cols = [x for x in df_seed.columns if "hhid" in x or "persid" in x]
        df_seed = df_seed.drop(columns=id_cols)

        with open(os.path.join(processed_data, 'dict_hh_states.pickle'), 'rb') as handle:
            hh_state_names = pickle.load(handle)
        
        pool = get_pool(df_seed, hh_state_names)
        pool.to_csv("./pool_all_samples.csv", index=False)
    return pool

pool_ori = wrapper_get_pool(have_the_file=True)

In [3]:
len(pool_ori)

8597817

In [4]:
pool_ori.head()

,dwelltype,owndwell,hhinc,totalvehs,hhsize,Spouse,Child,Others,Grandchild
0,Terrace/Townhouse,Fully Owned,Nil income,1,2,1,0,0,0
1,Separate House,Being Purchased,2500-2999,2,3,1,1,0,0
2,Separate House,Being Purchased,1500-1749,2,2,1,0,0,0
3,Separate House,Fully Owned,Nil income,1,1,0,0,0,0
4,Flat or Apartment,Being Rented,800-999,0,1,0,0,0,0


In [5]:
census_data = pd.read_csv(os.path.join(data_dir, "hh_marginals_ipu.csv"), header=[0,1])

In [6]:
census_data.head()

zone_id        sample_geog hhsize                                 \
  Unnamed: 0_level_1 Unnamed: 1_level_1      1      2     3     4     5     6   
0               3000                  2  11981  10112  3614  2244  1706  1616   
1               3002                  2   1170   1154   327   236   173   144   
2               3003                  2   1672   1760   646   392   208   179   
3               3004                  2   2871   2687   816   533   338   311   
4               3006                  2   5811   5732  1979  1113   758   712   

               ...      hhinc totalvehs                          \
      7    8+  ... Nil income         0     1     2     3    4+   
0  1592  1591  ...       3208     18125  8062  3077  2596  2596   
1   136   137  ...        149       672  1512   762   280   251   
2   176   170  ...        297      1671  2239   673   331   289   
3   310   305  ...        418      1693  4066  1330   575   507   
4   707   702  ...        988      5965  7155  2072  1187  1135   

         owndwell                                          
  Being Purchased Being Rented Fully Owned Something Else  
0            2682        17983        2515          11276  
1             451         1381         692            953  
2             820         2678         484           1221  
3            1255         3489        1316           2111  
4            2216         8565        1612           5121  

[5 rows x 45 columns]

- Now need to build the tree, to store number of each combinations (for adjustments)
- Thinking deeper, if we have the tree struct, we know that a combination exist or not
- Can we sample from a tree? There should be a way already for storing distributions as tree then sample it
- BN is kinda like that, can we link from BN to here?

In [7]:
# Let's just build the tree first
ALL_RELA_CONSI = ["Others", "Spouse", "Child", "Grandchild"] 
ordered_atts = list(pool_ori.columns)
for re in ALL_RELA_CONSI:
    ordered_atts.remove(re)
ordered_atts

['dwelltype', 'owndwell', 'hhinc', 'totalvehs', 'hhsize']

In [8]:
def building_counts_tree(check_df, ordered_atts, i_att):
    att = ordered_atts[i_att]
    hold_vals = check_df[att].value_counts()
    dict_sub_trees = {}
    next_i_att = i_att + 1
    for state in hold_vals.index:
        val = hold_vals[state]
        sub_new_df = check_df[check_df[att]==state]
        if next_i_att >= len(ordered_atts):
            children = None
        else:
            children = building_counts_tree(sub_new_df, ordered_atts, next_i_att)
        dict_sub_trees[f"{att}_{state}"] = {
            "lev": next_i_att,
            "count": val,
            "children": children
            }
    return dict_sub_trees

    # The thing here is I should return a list of dict or dict of dict
tree_store = {
    "lev": 0,
    "count": len(pool_ori),
    "children": building_counts_tree(pool_ori, ordered_atts, i_att=0)
}

In [9]:
tree_store

{'lev': 0,
 'count': 8597817,
 'children': {'dwelltype_Separate House': {'lev': 1,
   'count': 5923751,
   'children': {'owndwell_Fully Owned': {'lev': 2,
     'count': 2958953,
     'children': {'hhinc_2500-2999': {'lev': 3,
       'count': 290317,
       'children': {'totalvehs_2': {'lev': 4,
         'count': 128279,
         'children': {'hhsize_2': {'lev': 5, 'count': 72019, 'children': None},
          'hhsize_4': {'lev': 5, 'count': 24885, 'children': None},
          'hhsize_3': {'lev': 5, 'count': 19860, 'children': None},
          'hhsize_5': {'lev': 5, 'count': 6129, 'children': None},
          'hhsize_1': {'lev': 5, 'count': 4543, 'children': None},
          'hhsize_6': {'lev': 5, 'count': 668, 'children': None},
          'hhsize_8+': {'lev': 5, 'count': 102, 'children': None},
          'hhsize_7': {'lev': 5, 'count': 73, 'children': None}}},
        'totalvehs_1': {'lev': 4,
         'count': 99370,
         'children': {'hhsize_1': {'lev': 5, 'count': 42096, 'childre

Maybe a way to adjust the total count (probs optimise by having another struct like a dict to store the totals counts)
Now need to build an algorithm to update the sub tree with each total update (decrease or increase) that best maintains the current dist. We will start from the pool tree and update at each level with the value from census.
Need to build an algorithm to allocate the adjustment for each att to each sub_tree
For each adjustment we will have a list of sub_trees with the first level to be the current adjusting lev

Need to think a bit
We can find to what extent a cluster (i.e. sub-tree) can be adjusted for each deleted state(max can delete will the count of that state and max add will be the available plus state with max to be the count of parent minus the state don't need to be adjust, whichever smaller number would be the max we can adjust). Now to decide which cluster to delete we can based on the current dist of what to delete (more possible to delete we will delete more)
To balance out (as we will equally divide), we can use mod then take the number minus mod and divide and use the mod to plus 1 for n with n = mod

Bit note, end resulst of this we can have the tree_struct with only hh atts (not the rela). We need to connect again by doing sampling again for each combinations (will this increase the runtime even more?)

In [10]:
def adjust_sum_of_tree(root_tree, adjust_val):
    old_pa_val = root_tree["count"]
    root_tree["count"] = root_tree["count"] + adjust_val
    children = root_tree["children"]
    if children is not None:
        # Now will will update based on the distribution of the children
        for key, child_tree in children.items():
            # print(key, child_tree["count"])
            child_adjust_val = (child_tree["count"]/old_pa_val)*adjust_val
            adjust_sum_of_tree(child_tree, child_adjust_val)


test_tree = copy.deepcopy(tree_store)
re_tree = adjust_sum_of_tree(test_tree, -100000)
test_tree

{'lev': 0,
 'count': 8497817,
 'children': {'dwelltype_Separate House': {'lev': 1,
   'count': 5854852.685462717,
   'children': {'owndwell_Fully Owned': {'lev': 2,
     'count': 2924537.833917726,
     'children': {'hhinc_2500-2999': {'lev': 3,
       'count': 286940.36381432635,
       'children': {'totalvehs_2': {'lev': 4,
         'count': 126787.0049970824,
         'children': {'hhsize_2': {'lev': 5,
           'count': 71181.35714251653,
           'children': None},
          'hhsize_4': {'lev': 5,
           'count': 24595.566065781582,
           'children': None},
          'hhsize_3': {'lev': 5,
           'count': 19629.011133872704,
           'children': None},
          'hhsize_5': {'lev': 5, 'count': 6057.714463217814, 'children': None},
          'hhsize_1': {'lev': 5, 'count': 4490.161006101897, 'children': None},
          'hhsize_6': {'lev': 5, 'count': 660.2305859731604, 'children': None},
          'hhsize_8+': {'lev': 5,
           'count': 100.81365234919515,
 

In [25]:
# we need a ref dict of value from census
census_data = pd.read_csv(os.path.join(data_dir, "hh_marginals_ipu.csv"), header=[0,1])

def get_dict_vals(census_df):
    census_df = census_df.set_index(census_df.columns[census_df.columns.get_level_values(0)=="zone_id"][0])
    re_dict = {}
    ls_atts = census_df.columns.get_level_values(0).unique()
    for att in ls_atts:
        if att not in ["zone_id", "sample_geog"]:
            re_dict[att] = {}
            ls_idx = census_df.columns[census_df.columns.get_level_values(0)==att]
            ls_states = [x[1] for x in ls_idx]
            for state in ls_states:
                re_dict[att][state] = census_df[(att, state)]
    return re_dict

dict_census_vals = get_dict_vals(census_data)

In [ ]:
# Now we need to continue to have the algorithm for comparing with census and make adjustment
# First we need to have a way to loop through the tree and get the adjustable value


def test_func(root_tree, ref_dict):
    old_pa_val = root_tree["count"]
    children = root_tree["children"]
    if children is not None:
        # Now will will update based on the distribution of the children
        for key, child_tree in children.items():
            # print(key, child_tree["count"])
            test_func(child_tree)